In [9]:
from datetime import date

import numpy as np
import pandas as pd
import yaml

In [10]:
from sqlalchemy import create_engine

with open('./config.yml', 'r') as f:
        config = yaml.safe_load(f)
        config_ryf = config['RAPIDO_Y_FURIOSO']
        config_etl = config['ETL']

url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@{config_ryf['host']}:"
        f"{config_ryf['port']}/{config_ryf['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
        f"{config_etl['port']}/{config_etl['dbname']}")

ryf = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

In [11]:
dim_ciudad = pd.read_sql_table('ciudad', ryf)
departamento = pd.read_sql_table('departamento', ryf)

In [12]:
dim_ciudad = dim_ciudad.merge(
    departamento,
    on='departamento_id',
    how='left'
)

In [15]:
dim_ciudad = dim_ciudad.rename(columns={
    'nombre_x': 'nombre_ciudad',
    'nombre_y': 'nombre_departamento'
})

In [16]:
dim_ciudad.head()

,ciudad_id,nombre_ciudad,departamento_id,nombre_departamento
0,6,BUGA,1,VALLE DEL CAUCA
1,5,BOGOTA,2,CUNDINAMARCA
2,4,PASTO,4,NARIÑO
3,3,POPAYAN,3,CAUCA
4,2,PALMIRA,1,VALLE DEL CAUCA


In [17]:
dim_ciudad.to_sql('dim_ciudad', etl_conn, if_exists='replace',index_label='key_dim_ciudad')

19